In [1]:
import shutil
import optparse
from os import listdir, mkdir
from os.path import abspath, join, exists
from codecs import open
from lxml import etree
from math import floor
import glob

In [2]:
from pathlib import Path
from tqdm import tqdm

In [3]:
import pandas as pd
import os,sys

### 整合数据集

In [6]:
import shutil
def copy_data(plist, mode='copy'):
    ps = plist[0]
    pt = plist[1]
    if mode=='move':
        shutil.move(ps, pt)
    else:
        shutil.copyfile(ps, pt)

In [162]:
names = {0: ['小西装', 'Suit_xizhuang'],  
 1: ['毛衣/绒衣/针织衫', 'Sweater/knit'],   
 2: ['女衫类', 'Blouses '],   
 3: ['皮衣外套', 'Leather coat'],   
 4: ['夹克外套', 'Jacket/coat '],    
 5: ['风衣', 'dust coat'],    
 6: ['背心', 'vest'],    
 7: ['吊带', 'harness'],    
 8: ['牛仔外套', 'Denim jacket'],    
 9: ['羽绒服', 'Down jacket'],    
 10: ['大衣', 'coat'],   
 11: ['皮草', 'fur'],    
 12: ['卫衣', 'fleece'],     
 13: ['棉衣/羊羔绒', 'Padded coat'],    
 14: ['斗篷', 'cloak'],    
 15: ['半身裙', 'skirt'],
 16: ['西装裤', 'Suit pants'],    
 17: ['牛仔裤', 'Jeans'],     
 18: ['休闲裤', 'Slacks'],      
 19: ['打底裤', 'Leggings'],    
 20: ['运动裤', 'Sweatpants'],
21: ['连衣裙', 'Dress'],     
 22: ['连衣裤', 'Jumpsuit'],    
 23: ['正装女鞋', 'Suits shoes'],   
 24: ['帆布鞋', 'Canvas shoes'],     
 25: ['松糕鞋', 'Platform shoes'],    
 26: ['洞洞鞋', 'Hole shoes'],     
 27: ['懒人鞋', 'Lazy shoes'],     
 28: ['板鞋', 'board shoes'],    
 29: ['拖鞋', 'slippers'],     
 30: ['凉鞋', 'sandals'],    
 31: ['运动鞋', 'Sports shoes'],    
 32: ['雪地靴', 'Ugg boots'],   
 33: ['豆豆鞋', 'Doug shoes'],    
 34: ['网鞋', 'Net shoes'],      
35: ['包', 'bag'],   
36:['衬衫', 'shirt'],  
37:['T恤', 'T-shirt'],
38:['POLO衫','POLO shirt'],
39:['马甲', 'Vest'] ,
40:['棒球服', 'Baseball uniform'], 
41:['抹胸', 'Bandeau'],
42:['皮裤','Leather pants'],
43:['休闲鞋','Leisure shoes'],
44:['帽子','Hat'],}

### XML

In [7]:
def voc2yolo(output_p, imgs_p, labels_p, split_ratios = None):
    img_filetype = ['.jpg',  '.jpeg', '.png', '.tiff', '.tif']
    classes_name = {} # store unique class name {class_name:0}
    yolo_labels = {} # {file_name:file_content}
    xmlFiles = list(filter(lambda x: x.endswith('.xml'), listdir(labels_p)))
    n_imgs = len(xmlFiles)
    counter = 0
    for xmlFile in xmlFiles:
        trees = etree.parse(labels_p + '/' + xmlFile)
        width, height = int(trees.find('size/width').text), int(trees.find('size/height').text)
        yolo_labels[xmlFile[:-4]] = ''
        for obj in trees.findall('object'):
            bdb = obj.find('bndbox')
            name = obj.find('name').text
            xmin = int(bdb.find('xmin').text)
            ymin = int(bdb.find('ymin').text)
            xmax = int(bdb.find('xmax').text)
            ymax = int(bdb.find('ymax').text)
            center_x = (xmin + (xmax-xmin)/2)/width
            center_y = (ymin + (ymax-ymin)/2)/height
            bdb_w = (xmax-xmin)/width
            bdb_h = (ymax-ymin)/height
            if name not in classes_name.keys():
                classes_name[name] = len(classes_name)
            yolo_labels[xmlFile[:-4]] += '{} {:6f} {:6f} {:6f} {:6f}\n'.format(
                    classes_name[name], center_x, center_y, bdb_w, bdb_h)
    if split_ratios == None:
        for filename in yolo_labels:
            save_path = join(output_p, 'labels')
            open(join(save_path, filename+'.txt'), 'w', encoding='utf-8').write(yolo_labels[filename])
            for ft in img_filetype:
                src = join(imgs_p, f'{filename}{ft}')
                dst = join(output_p, f'images/{filename}{ft}')
                if exists(src):
                    shutil.copy(src, dst)
                    break
    else:
        n_train = floor(n_imgs*(split_ratios[0]/10))
        n_val = round(n_train*(split_ratios[2]/10))
        for filename in yolo_labels:
            if counter < n_train:
                if counter < n_train - n_val:
                    split_to = 'train'
                else:
                    split_to = 'val'
            else:
                split_to = 'test'
            save_path = join(output_p, f'labels/{split_to}')
            open(join(save_path, f'{filename}.txt'), 'w', encoding='utf-8').write(yolo_labels[filename])
            for ft in img_filetype:
                src = join(imgs_p, f'{filename}{ft}')
                dst = join(output_p, f'images/{split_to}/{filename}{ft}')
                if exists(src):
                    shutil.copy(src, dst)
                    break
            counter += 1
    with open(f'{output_p}/dataset_meta.txt', 'w', encoding='utf-8') as f:
        content = 'Class Name\tID\n'
        for class_name in classes_name:
            content += f'{class_name}\t{classes_name[class_name]}\n'
        if split_ratios != None:
            content += f'Number of Training Set : {floor(n_imgs*(split_ratios[0]/10))-round(floor(n_imgs*(split_ratios[0]/10))*(split_ratios[2]/10))}\nNumber of Testing Set : {counter-floor((n_imgs*(split_ratios[0]/10)))}\nNumber of Validation Set : {round(floor(n_imgs*(split_ratios[0]/10))*(split_ratios[2]/10))}'
        else:
            content += f'Number of Dataset : {n_imgs}'
        f.write(content)
    print('Convert completed!')

In [8]:
def voc2yolo_xml(output_p, labels_p, split_ratios = None):
    classes_name = {} # store unique class name {class_name:0}
    yolo_labels = {} # {file_name:file_content}
    xmlFiles = list(filter(lambda x: x.endswith('.xml'), listdir(labels_p)))
    counter = 0
    COUNTER = 0
    for ind,xmlFile in tqdm(enumerate(xmlFiles)):
        trees = etree.parse(labels_p + '/' + xmlFile)
        width, height = int(trees.find('size/width').text), int(trees.find('size/height').text)
        yolo_labels[xmlFile[:-4]] = ''
        if width*height==0:
            # print(xmlFile, width, height)
            COUNTER += 1
            continue

        for obj in trees.findall('object'):
            bdb = obj.find('bndbox')
            name = obj.find('name').text
            xmin = int(bdb.find('xmin').text)
            ymin = int(bdb.find('ymin').text)
            xmax = int(bdb.find('xmax').text)
            ymax = int(bdb.find('ymax').text)
            center_x = (xmin + (xmax-xmin)/2)/width
            center_y = (ymin + (ymax-ymin)/2)/height
            bdb_w = (xmax-xmin)/width
            bdb_h = (ymax-ymin)/height
            if name not in classes_name.keys():
                classes_name[name] = len(classes_name)
            yolo_labels[xmlFile[:-4]] += '{} {:6f} {:6f} {:6f} {:6f}\n'.format(
                    classes_name[name], center_x, center_y, bdb_w, bdb_h)
    return yolo_labels

In [ ]:
labels_p = Path('../../dataset/haowei34k_labelImg/annotations_labelme/')
output_p = Path('../../dataset/haowei34k_labelImg/labels_voc')
Path.mkdir(output_p, parents=True, exist_ok=True)

In [ ]:
import re

In [ ]:
classes_name = {} # store unique class name {class_name:0}
yolo_labels = {} # {file_name:file_content}
# xmlFiles = list(filter(lambda x: x.endswith('.xml'), listdir(labels_p)))
xml_list = glob.glob(str(labels_p /'*/*.xml'))
xmlFiles = [re.findall(r'/[0-9]{1,3}/(.*?.xml)',x)[0] for x in xml_list]
len(xmlFiles)

##### voc2yolo

In [ ]:
counter = 0
COUNTER = 0
xml_erros = []

In [ ]:
classes_name = {} # store unique class name {class_name:0}
yolo_labels = {} # {file_name:file_content}
# xmlFiles = list(filter(lambda x: x.endswith('.xml'), listdir(labels_p)))
xml_list = glob.glob(str(labels_p /'*/*.xml'))
xmlFiles = [re.findall(r'/([0-9]{1,3}/.*?.xml)',x)[0] for x in xml_list]

for ind,xmlFile in tqdm(enumerate(xmlFiles)):
    trees = etree.parse(labels_p /xmlFile)
    width, height = int(trees.find('size/width').text), int(trees.find('size/height').text)
    if width*height==0:
        # print(xmlFile, width, height)
        xml_erros.append(xmlFile)
        COUNTER += 1
        continue
    fname = re.findall(r'[0-9]{1,3}/(.*?).xml',xmlFile)[0]
    yolo_labels[fname] = ''
        
    for obj in trees.findall('object'):
        bdb = obj.find('bndbox')
        name = obj.find('name').text
        xmin = int(bdb.find('xmin').text)
        ymin = int(bdb.find('ymin').text)
        xmax = int(bdb.find('xmax').text)
        ymax = int(bdb.find('ymax').text)
        center_x = (xmin + (xmax-xmin)/2)/width
        center_y = (ymin + (ymax-ymin)/2)/height
        bdb_w = (xmax-xmin)/width
        bdb_h = (ymax-ymin)/height
        if name not in classes_name.keys():
            classes_name[name] = len(classes_name)
        yolo_labels[fname] += '{} {:6f} {:6f} {:6f} {:6f}\n'.format(
                classes_name[name], center_x, center_y, bdb_w, bdb_h)

### get voc

In [13]:
import re

In [16]:
labels_p = Path('../../dataset/haowei34k_labelImg/haowei34k_catg45/xml/')
xml_list = glob.glob(str(labels_p /'*.xml'))
xmlFiles = [re.findall(r'/xml/(.*?.xml)',x)[0] for x in xml_list]

In [17]:
xml_list[0]

'../../dataset/haowei34k_labelImg/haowei34k_catg45/xml/20160425101358677.xml'

In [18]:
xmlFiles[0]

'20160425101358677.xml'

In [22]:
classes_name = {} # store unique class name {class_name:0}
yolo_labels = {} # {file_name:file_content}
# xmlFiles = list(filter(lambda x: x.endswith('.xml'), listdir(labels_p)))



for ind,xmlFile in tqdm(enumerate(xmlFiles)):
    trees = etree.parse(labels_p /xmlFile)
    width, height = int(trees.find('size/width').text), int(trees.find('size/height').text)
    # fname = re.findall(r'[0-9]{1,3}/(.*?).xml',xmlFile)[0]
    fname = xmlFile[:-4]
    yolo_labels[fname] = ''
        
    for obj in trees.findall('object'):
        bdb = obj.find('bndbox')
        name = obj.find('name').text
        xmin = int(bdb.find('xmin').text)
        ymin = int(bdb.find('ymin').text)
        xmax = int(bdb.find('xmax').text)
        ymax = int(bdb.find('ymax').text)
        # center_x = (xmin + (xmax-xmin)/2)/width
        # center_y = (ymin + (ymax-ymin)/2)/height
        # bdb_w = (xmax-xmin)/width
        # bdb_h = (ymax-ymin)/height
        if name not in classes_name.keys():
            classes_name[name] = len(classes_name)
        yolo_labels[fname] += '{} {:6f} {:6f} {:6f} {:6f}\n'.format(
                classes_name[name], xmin, ymin, xmax, ymax)
    # break

33411it [00:01, 18600.87it/s]


In [ ]:
# write labels text.
for filename in tqdm(yolo_labels.keys()):
    save_path = join(output_p, 'labels')
    open(join(save_path, filename+'.txt'), 'w', encoding='utf-8').write(yolo_labels[filename])
    # break

In [ ]:
txt_df = pd.DataFrame()

In [ ]:
txt_df['fname']  = yolo_labels.keys()

In [ ]:
fname_df = pd.DataFrame()
fname_df['fname'] = fname_list

In [ ]:
empty_df = pd.DataFrame()
empty_df = errors_df[~(errors_df['errors_fname'].isin(txt_df['fname']))]

In [ ]:
fname_df['fname'].drop_duplicates().info()

In [ ]:
len(yolo_labels.keys()),len(fname_list)

In [ ]:
# p = labels_p + '/' + xml_erros[0]
name = filename+'.txt'
p = Path(save_path) / name
!cat $p

### 解析json文件

In [ ]:
!ls /home/chenyi/workspace/dataset/haowei34k_labelImg/

In [ ]:
!ls /home/chenyi/workspace/dataset/haowei34k_labelImg/annotations_labelme/

In [ ]:
json_list = glob.glob('/home/chenyi/workspace/dataset/haowei34k_labelImg/xml_label/*/*.json')

In [ ]:
json_df = pd.DataFrame()
json_df['spath'] = json_list

In [ ]:
xml_dir = Path('/home/chenyi/workspace/dataset/haowei34k_labelImg/json2xml/')

In [ ]:
json_df['tpath'] = json_df['spath'].apply(lambda x:
                                         '{0}/{1}'.format(xml_dir, re.findall(r'/[0-9]{1,3}/(.*?.json)',x)[0]))

In [ ]:
json_df[1:3].values

In [ ]:
# input_list = list(zip(json_df['spath'], json_df['tpath']))
# for ps in tqdm(input_list):
#     copy_data(ps)

In [ ]:
# p = '/home/chenyi/workspace/dataset/haowei34k_labelImg/json2xml/20150409182919078.json'
# !cat $p


In [ ]:
sys.path.append('/home/chenyi/workspace/myPyModule99/data_format/json_to_xml')
from json_to_xml.main import *

In [ ]:
root_json_dir = xml_dir
root_save_xml_dir = '/home/chenyi/workspace/dataset/haowei34k_labelImg/json2xml/xml'

In [ ]:
import json
for json_filename in tqdm(os.listdir(root_json_dir)):
    json_path = os.path.join(root_json_dir, json_filename)
    json_data = json.load(open(json_path))
    break

In [ ]:
json_data.keys()

In [ ]:
json_data['shapes'][0]['points']

In [ ]:
json_df = pd.DataFrame()

In [ ]:
root_json_dir

In [ ]:
json_df['path'] = glob.glob(str(root_json_dir/'*.json'))

In [ ]:
json_df['fname'] =  json_df['path'].apply(lambda x: re.findall(r'/json2xml/(.*?).json',x)[0])

In [ ]:
json_labels = {}
for ind,row in tqdm(json_df.iterrows()):
    json_path = row['path']
    fname = row['fname']
    json_data = json.load(open(json_path))
    json_labels[fname] = ''
    for item in json_data['shapes']:
        name = item['label']
        points = item['points']
        xmin, ymin = points[0]
        xmax, ymax = points[1]
        
        if name not in classes_name.keys():
            classes_name[name] = len(classes_name)
        json_labels[fname] += '{} {:6f} {:6f} {:6f} {:6f}\n'.format(
            classes_name[name], xmin, ymin, xmax, ymax)

In [ ]:
fname_list = []
xmin_list = []
ymin_list = []
xmax_list = []
ymax_list = []
labels_list = []
for ind,row in tqdm(json_df.iterrows()):
    json_path = row['path']
    fname = row['fname']
    json_data = json.load(open(json_path))
    for item in json_data['shapes']:
        name = item['label']
        points = item['points']
        xmin, ymin = points[0]
        xmax, ymax = points[1]
        
        fname_list.append(fname)
        labels_list.append(name)
        xmin_list.append(xmin)
        ymin_list.append(ymin)
        ymax_list.append(ymax)
        xmax_list.append(xmax)

In [ ]:
json_bboxes = pd.DataFrame()

In [ ]:
json_bboxes['fname'] = fname_list
json_bboxes['label'] = labels_list
json_bboxes['xmin'] = xmin_list
json_bboxes['ymin'] = ymin_list
json_bboxes['xmax'] = xmax_list
json_bboxes['ymax'] = ymax_list

In [ ]:
json_bboxes.head()

### pandas处理

In [23]:
labels_p

PosixPath('../../dataset/haowei34k_labelImg/haowei34k_catg45/xml')

In [24]:
xml_list = glob.glob(str(labels_p /'*.xml'))
xmlFiles = [re.findall(r'/xml/(.*?.xml)',x)[0] for x in xml_list]
fname_list = []
xmin_list = []
ymin_list = []
xmax_list = []
ymax_list = []
labels_list = []
for ind,xmlFile in tqdm(enumerate(xmlFiles)):
    trees = etree.parse(labels_p /xmlFile)
    width, height = int(trees.find('size/width').text), int(trees.find('size/height').text)
    # fname = re.findall(r'[0-9]{1,3}/(.*?).xml',xmlFile)[0]     
    fname = xmlFile[:-4]     

    for obj in trees.findall('object'):
        bdb = obj.find('bndbox')
        name = obj.find('name').text
        xmin = int(bdb.find('xmin').text)
        ymin = int(bdb.find('ymin').text)
        xmax = int(bdb.find('xmax').text)
        ymax = int(bdb.find('ymax').text)
        
        fname_list.append(fname)
        labels_list.append(name)
        xmin_list.append(xmin)
        ymin_list.append(ymin)
        ymax_list.append(ymax)
        xmax_list.append(xmax)

33411it [00:01, 20297.55it/s]


In [25]:
xml_bboxes = pd.DataFrame()
xml_bboxes['fname'] = fname_list
xml_bboxes['label'] = labels_list
xml_bboxes['xmin'] = xmin_list
xml_bboxes['ymin'] = ymin_list
xml_bboxes['xmax'] = xmax_list
xml_bboxes['ymax'] = ymax_list

In [26]:
xml_bboxes.head()

,fname,label,xmin,ymin,xmax,ymax
0,20160425101358677,正装女鞋,225,584,258,669
1,20160425101358677,正装女鞋,256,672,305,768
2,20160425101358677,连衣裙,157,136,357,468
3,20160425101358677,包,118,135,222,413
4,20170818201332676,牛仔,447,308,745,767


In [27]:
xml_bboxes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144122 entries, 0 to 144121
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   fname   144122 non-null  object
 1   label   144122 non-null  object
 2   xmin    144122 non-null  int64 
 3   ymin    144122 non-null  int64 
 4   xmax    144122 non-null  int64 
 5   ymax    144122 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 6.6+ MB


In [28]:
labelme_df = xml_bboxes

In [ ]:
# labelme_df.to_csv('/home/chenyi/workspace/dataset/haowei34k_labelImg/labelme_hw34k.csv', index=False)

In [30]:
labelme_df['label'].value_counts().axes,len(labelme_df['label'].value_counts().axes[0])

([Index(['正装女鞋', '正装女鞋（宽容型）', '包', '裙子', '休闲裤', '毛衣/绒衣/针织衫', '牛仔裤', '休闲裤（宽容型）',
         '连衣裙', '连衣裙（严格型）',
         ...
         '正装你学', '毛衣/绒衣/针织衫衫', '版鞋', '毛衣/绒衣//针织衫', '/外套', '板鞋\t', '女衫 ', '羊羔毛',
         '牛', '衬衫 '],
        dtype='object', length=149)],
 149)

In [31]:
import pickle
with open('labelme_names_dict.pickle', 'rb') as f:
    names_dict = pickle.load(f)

In [53]:
# import pickle
# # with open('labelme_names_dict.pickle', 'wb') as f:
#     pickle.dump(names_dict, f)

In [52]:
labelme_df['label43'] = labelme_df['label'].apply(lambda x: names_dict[x])

In [63]:
labelme_df.to_csv('labelme_voc_bboxes_relabel.csv', index=False)

In [55]:
labelme_df['label43'].value_counts()

正装女鞋         44481
包            20515
休闲裤           8833
半身裙           7625
毛衣/绒衣/针织衫     7225
连衣裙           6941
牛仔裤           6875
女衫类           4035
帆布鞋           3767
凉鞋            3225
板鞋            2888
小西装           2670
夹克外套          2355
运动鞋           2182
背心            2180
皮衣外套          1741
拖鞋            1495
卫衣            1490
大衣            1449
打底裤           1305
懒人鞋           1294
牛仔外套          1046
西装裤           1001
T恤             932
豆豆鞋            836
松糕鞋            663
衬衫             576
皮裤             522
吊带             504
连衣裤            426
雪地靴            422
风衣             417
棉衣/羊羔绒         386
马甲             376
休闲鞋            343
皮草             321
羽绒服            246
运动裤            166
帽子             156
网鞋              60
抹胸              47
棒球服             46
斗篷              36
洞洞鞋             12
POLO衫           11
Name: label43, dtype: int64

In [76]:
root = Path('/home/chenyi/workspace/dataset/haowei34k/')

In [77]:
imgs_list = glob.glob(str(root /'images/*/*.jpg'))

In [78]:
imgs_df = pd.DataFrame()
imgs_df['path'] = imgs_list

In [98]:
import re

In [102]:
imgs_df['fname']= imgs_df['path'].apply(lambda x: re.findall(r'images/.*?/(.*?).jpg', x)[0])

In [103]:
imgs_df.head(2).values

array([['/home/chenyi/workspace/dataset/haowei34k/images/train/20160406001119728.jpg',
        '20160406001119728'],
       ['/home/chenyi/workspace/dataset/haowei34k/images/train/20141110184352626.jpg',
        '20141110184352626']], dtype=object)

In [ ]:
_ = [os.symlink(p, p.replace()) for p in imgs_list]

In [114]:
labelme_df['fname'] = labelme_df['fname'].astype(str)

In [120]:
imgs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34020 entries, 0 to 34019
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    34020 non-null  object
 1   fname   34020 non-null  object
dtypes: object(2)
memory usage: 531.7+ KB


In [115]:
labelme_df.drop_duplicates('fname').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33411 entries, 0 to 144118
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   fname    33411 non-null  object
 1   label    33411 non-null  object
 2   xmin     33411 non-null  int64 
 3   ymin     33411 non-null  int64 
 4   xmax     33411 non-null  int64 
 5   ymax     33411 non-null  int64 
 6   label43  33411 non-null  object
dtypes: int64(4), object(3)
memory usage: 2.0+ MB


In [110]:
x = labelme_df[~labelme_df['fname'].isin(imgs_df['fname'])]

In [116]:
y = imgs_df[~imgs_df['fname'].isin(labelme_df['fname'])]

In [122]:
y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 609 entries, 194 to 33973
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    609 non-null    object
 1   fname   609 non-null    object
dtypes: object(2)
memory usage: 14.3+ KB


In [125]:
labelme_df.head()

,fname,label,xmin,ymin,xmax,ymax,label43
0,20160425101358677,正装女鞋,225,584,258,669,正装女鞋
1,20160425101358677,正装女鞋,256,672,305,768,正装女鞋
2,20160425101358677,连衣裙,157,136,357,468,连衣裙
3,20160425101358677,包,118,135,222,413,包
4,20170818201332676,牛仔,447,308,745,767,牛仔外套


In [127]:
imgs_df.head(2).values

array([['/home/chenyi/workspace/dataset/haowei34k/images/train/20160406001119728.jpg',
        '20160406001119728'],
       ['/home/chenyi/workspace/dataset/haowei34k/images/train/20141110184352626.jpg',
        '20141110184352626']], dtype=object)

In [132]:
imgs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34020 entries, 0 to 34019
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    34020 non-null  object
 1   fname   34020 non-null  object
dtypes: object(2)
memory usage: 531.7+ KB


In [123]:
root

PosixPath('/home/chenyi/workspace/dataset/haowei34k')

In [58]:
from PIL import Image

In [128]:
w_list = []
h_list = []
for ind, row in tqdm(imgs_df.iterrows()):
    p_ = row['path']
    try:
        img = Image.open(str(p_))
        w,h = img.size
    except:
        w,h = 0,0
        print(p_)
    w_list.append(w)
    h_list.append(h)  

34020it [00:06, 5175.93it/s]


In [133]:
imgs_df['width'] = w_list
imgs_df['height'] = h_list

In [139]:
imgs_df['mode'] = imgs_df['path'].apply(lambda x: re.findall(r'/images/([a-z]{3,5})/', x)[0])

In [143]:
val_df = imgs_df[imgs_df['mode']=='val']

In [144]:
labelme_df['mode'] = 'train'

In [150]:
labelme_df.loc[labelme_df[labelme_df['fname'].isin(val_df['fname'])].index,'mode'] = 'val'

In [155]:
w_list = []
h_list = []
for ind, row in tqdm(labelme_df.iterrows()):
    fname = row['fname']
    mode = row['mode']
    p_ = f'/home/chenyi/workspace/dataset/haowei34k/images/{mode}/{fname}.jpg'
    try:
        img = Image.open(str(p_))
        w,h = img.size
    except:
        w,h = 0,0
        print(p_)
    w_list.append(w)
    h_list.append(h)  
    # break

144122it [00:07, 18863.25it/s]


In [156]:
labelme_df['width'] = w_list
labelme_df['height'] = h_list

In [163]:
names_lab2num = {}
for k,v in names.items():
    names_lab2num[v[0]] = k

In [168]:
labelme_df.to_csv('haowei34k_voc_bboxes_relabel.csv', index=False)

In [166]:
labelme_df['label_num'] = labelme_df['label43'].apply(lambda x:
                                                     names_lab2num[x])

In [159]:
labelme_df.describe()

,xmin,ymin,xmax,ymax,width,height
count,144122.000000,144122.000000,144122.000000,144122.000000,144122.000000,144122.000000
mean,250.523896,519.374481,385.157339,731.878110,640.558131,937.019539
std,152.861918,403.082670,214.729285,430.394688,332.120837,486.387922
min,0.000000,0.000000,34.000000,27.000000,173.000000,214.000000
25%,161.000000,235.000000,263.000000,484.000000,450.000000,657.000000
50%,216.000000,452.000000,336.000000,637.000000,599.000000,852.000000
75%,293.000000,701.000000,434.000000,860.000000,720.000000,1000.000000
max,3013.000000,5513.000000,3534.000000,6031.000000,4195.000000,6286.000000


In [175]:
labelme_df[labelme_df['height']<300]

,fname,label,xmin,ymin,xmax,ymax,label43,mode,width,height,label_num
6211,20150409155840687,连衣裙（严格型）,71,46,170,204,连衣裙,train,210,280,21
6212,20150409155840687,包,117,95,179,162,包,train,210,280,35
19330,20140305214501056,皮衣,59,56,129,130,皮衣外套,train,200,299,3
19331,20140305214501056,休闲裤,76,124,123,265,休闲裤,train,200,299,18
19332,20140305214501056,包,81,151,134,203,包,train,200,299,35
...,...,...,...,...,...,...,...,...,...,...,...
136950,20141022224902760,小西装（严格型）,58,43,135,144,小西装,train,190,280,0
136951,20141022224902760,休闲裤（宽容型）,69,131,122,150,休闲裤,train,190,280,18
136952,20141022224902760,包,48,151,82,233,包,train,190,280,35
136953,20141022224902760,正装女鞋（宽容型）,78,234,93,278,正装女鞋,train,190,280,23
